# Standardize and Normalize Dashlink Telemetry file

This is a code to standardize and normalize the time series data by time stamp into z-scores.

## S3 Data read logic
This is standard functions to read bucket object keys for retrieving the data

In [1]:
from scipy import io
import pandas as pd
import numpy as np
import csv
import matplotlib as mp
import matplotlib.pyplot as plt
from numpy.lib import recfunctions
import datetime
import numpy.lib.recfunctions as recfn
import boto3
import sys
from IPython.display import clear_output
s3 = boto3.resource('s3')

In [2]:
def s3_bucket_object_keys(bucket_name= 'iiaweb-s3-io-practice-bucket'):
    bucket = s3.Bucket(bucket_name)
    key_list=[]
    for key in bucket.objects.filter(Prefix='652/1'):
        key_list.append(key)
    return(key_list)

In [3]:
from io import BytesIO
def load_file(s3_key, bucketname= 'iiaweb-s3-io-practice-bucket'):
    s3 = boto3.resource('s3')
    obj = s3.Object(bucketname, s3_key.key)
    inFile = obj.get()['Body'].read()
    gfile = BytesIO(inFile)
    return(gfile)

In [7]:
start = datetime.datetime.now()
key_list = s3_bucket_object_keys('iiaweb-s3-io-practice-bucket')
end = datetime.datetime.now()
print("Time Taken to run the funtion ",str(end-start))

Time Taken to run the funtion  0:00:00.291515


In [27]:
from scipy import stats
def standardize_normalize(key):
    '''
    Argument:
    key: S3 Bucket key for csv file
    Transformations:
    1. Read Data Frame
    2. Selecting intergers and Floats
    3. Filling Null Values
    4. Z-Score
    5. Drop Invalid values e.g. for near constant invariant fields
    Output: Valid Z score of Pandas Datafrane column
    '''
    gzfile= load_file(key)
    df = pd.read_csv(gzfile, compression='gzip')
    df1= df.select_dtypes(exclude=['object'])
    df1.fillna(0, inplace= True)
    df1= (df1 - df1.mean())/df1.std()
    df1.dropna(axis=1, inplace= True)
    return(df1)
    

## Example

In [28]:
key= key_list[0]

In [25]:
df2=standardize_normalize(key)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [26]:
df2.describe()

,EGT_1,EGT_2,EGT_3,EGT_4,MW,ALT,RALT,ALTR,FQTY_1,OIT_1,...,FLAP,PTRM,BPGR_2,BPYR_1,BPYR_2,OIPL,FIRE_2,FIRE_3,FIRE_4,FRMC
count,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,...,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03,3.943000e+03
mean,2.740221e-16,4.994117e-15,-7.781177e-15,-8.381381e-15,1.651499e-15,1.286423e-14,8.390631e-15,7.523500e-17,-1.263565e-15,1.396184e-15,...,-8.926328e-15,-2.474491e-16,-1.624071e-15,3.821907e-15,-6.879606e-16,5.494520e-16,-2.596322e-16,5.689400e-16,5.590394e-16,-1.000693e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.003396e+00,-8.780130e-01,-5.591412e-01,-3.026665e-01,-5.524386e-02,-1.984432e+00,-1.547819e+00,-6.465333e+00,-1.441202e+00,-1.533766e+01,...,-4.811735e+00,-2.217479e+01,-7.810003e+00,-8.416814e+00,-2.252458e-02,-9.984102e-01,-2.252458e-02,-3.186265e-02,-3.186265e-02,-1.728319e+00
25%,-1.043959e+00,-8.729192e-01,-5.557035e-01,-2.931127e-01,-5.524386e-02,-7.857473e-01,2.498394e-01,-8.503007e-01,-8.786365e-01,-4.270525e-01,...,-7.620719e-01,3.862661e-02,-1.377619e-01,-7.992431e-01,-2.252458e-02,-5.525743e-01,-2.252458e-02,-3.186265e-02,-3.186265e-02,-8.639405e-01
50%,8.504046e-01,-8.703723e-01,-5.557035e-01,-2.931127e-01,-5.524386e-02,-3.861856e-01,2.498394e-01,2.727057e-01,-3.478877e-02,2.071346e-02,...,-7.620719e-01,4.954778e-02,-1.377619e-01,-2.268244e-01,-2.252458e-02,-5.525743e-01,-2.252458e-02,-3.186265e-02,-3.186265e-02,4.384372e-04
75%,8.714141e-01,1.246101e+00,4.309058e-01,-2.883359e-01,-5.524386e-02,8.124996e-01,2.498394e-01,8.342089e-01,9.028198e-01,5.580322e-01,...,1.262759e+00,4.954778e-02,-1.377619e-01,1.006078e+00,-2.252458e-02,-1.067383e-01,-2.252458e-02,-3.186265e-02,-3.186265e-02,8.648174e-01
max,1.788832e+00,1.607761e+00,2.778830e+00,4.880232e+00,1.809697e+01,2.011185e+00,2.047498e+00,3.080222e+00,1.746667e+00,8.266916e-01,...,3.287591e+00,1.805998e-01,1.396687e+00,1.020755e+00,4.438468e+01,2.568278e+00,4.438468e+01,3.137675e+01,3.137675e+01,1.729196e+00
